Alaa Ashraf 20208007





Malak Medhat 20208031

Donia Essam 20208011

Mariam Ahmad 20208042


Menna Mahmoud 20208033

In [ ]:
!pip install pydicom

import os
import shutil
import sys
import numpy as np
import pandas as pd
import pydicom
import cv2


In [ ]:
csv_file = '/content/drive/MyDrive/yolov4/train_labels.csv'
df = pd.read_csv(csv_file)

After importing the necessary libraries, we read the train_labels.csv file and load it into a dataframe.

# Converting .dcm files to .jpg

In [ ]:
# Converting dcm images to jpg

train_path = '/content/drive/MyDrive/uni/dataset/train_images'
output_train_path = '/content/drive/MyDrive/uni/Output/train'

def convert(row,folder_path,output_path):
    file = os.path.join(folder_path, f"{row['patientId']}.dcm")
    print(file)
      # Check if the file exists
    if not os.path.isfile(file):
          print(f"File not found: {file}")
          return
    image = pydicom.dcmread(file) # reading the dcm file using dcmread
    image = image.pixel_array  # extracting the image pixels data from the read dcm
    image = cv2.resize(image, (224,224))  # resizing the images
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) # converting them to rgb
    out_path = os.path.join(output_path, f"{row['patientId']}.jpg")  # specifying the output path as the patientId.jpg
    cv2.imwrite(out_path, image)  # writing the image into the output path

for _ , row in df.iterrows(): # returns index and the content of each row
    convert(row,train_path,output_train_path)


We create a convert function which converts the dcm files to jpg by giving it a row from the train_labels.csv file, getting the patientId from it (which corresponds to a file name in the dcm folder), and converting it to a jpg file. We used dcmread from pydicom to convert it into an image, then we resize the images and convert them to RGB. We then create an image with the same patientId name using cv2 imwrite. We itterate over all the rows from the csv file and convert the corresponding images if patientId is in this way.


In [ ]:
images_folder = '/content/drive/MyDrive/data2/test'
dcm_path = '/content/drive/MyDrive/uni/dataset/test_images'

# Converting the dcm test files to jpg

def convert_test(file,output_path):
    print(file)
    image = pydicom.dcmread(file)
    image = image.pixel_array
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    out_path = os.path.join(output_path, f"{file.rstrip('.dcm')}.jpg")  # specifying the output file name by replacing the .dcm by .jpg of the original file
    cv2.imwrite(out_path, image)

for file_name in os.listdir(dcm_path):  # itterating over the files in the folder that contains the dcm test files
    file_path = os.path.join(dcm_path, file_name)
    convert_test(file_path,images_folder)


We itterate over the files in the folder that contains the dcm test images, converting them to jpg, resizing them and converting them to RGB, then saving them by replacing the .dcm to .jpg as the file names.


After creating the jpg images, we create a folder named yolov4. Inside it, we create another folder named dataset then copy the created train jpg images to this folder.

# Creating txt files inside the dataset folder

In [ ]:
train_images = '/content/drive/MyDrive/yolov4/dataset'

# Creating txt file for each image

# We will create bounding boxes where the target is 1
df_target_1 = df[df['Target'] == 1]

# Create empty text files where the target is 0
df_target_0 = df[df['Target'] == 0]

# Loop over each jpg image in train_images
for filename in os.listdir(train_images):
    if filename.endswith('.jpg'):

        # Get the patientId from the image filename
        patient_id = os.path.splitext(filename)[0]

        # Find the patientId in the dataframe for target=1
        rows_target_1 = df_target_1[df_target_1['patientId'] == patient_id]

        if not rows_target_1.empty:
            # Create txt file with bounding box data
            txt_filename = os.path.join(train_images, f"{patient_id}.txt")
            with open(txt_filename, 'w') as txt_file:
                for _, row in rows_target_1.iterrows():
                    # Target, x, y, width, height
                    target = row['Target']
                    x = row['x']
                    y = row['y']
                    width = row['width']
                    height = row['height']

                    # Content for the txt file (in that order)
                    content = f"{target} {x} {y} {width} {height}\n"
                    txt_file.write(content)

        # Find the patientId in the dataframe for target=0
        rows_target_0 = df_target_0[df_target_0['patientId'] == patient_id]

        if not rows_target_0.empty:
            # Create empty txt file
            txt_filename = os.path.join(train_images, f"{patient_id}.txt")
            with open(txt_filename, 'w') as txt_file:
                pass  # Create an empty file

We create two separate dataframes: one for targets = 1 and the other for targets = 0. For the df_target_1, we get the row with patientId similar to image name, create txt files with the same name, get it's row from the csv file and put inside the txt file the target and bounding box details in this format: Target, x, y, width, height. For the df_target_0, we create an empty txt file with the same name as the patientId.

In [ ]:
# Creating train.txt and test.txt

import os

path_list = []

# Path where the images are located
directory = '/content/drive/MyDrive/yolov4/dataset'
os.chdir(directory)

# Go through all the image files in the directory
for current_dir, dirs, files in os.walk(directory):
    # Iterating through all the files
    for f in files:
        # Check if the file extension ends with '.jpg'
        if f.endswith('.jpg'):
            # Prepare file path to save into train.txt
            file_loc = os.path.join(current_dir, f)
            # Append the path data into list "path_list"
            path_list.append(file_loc)


# Divide the data into 80:20 ratio. Get 20% of data from path_list to write into the test.txt file
path_list_test = path_list[:int(len(path_list) * 0.20)]

# Delete the same 20% records from the path_list as that 20% data is in path_list_test now
path_list = path_list[int(len(path_list) * 0.20):]

# Create train.txt file and write 80% of data (lines) inside it.
with open('train.txt', 'w') as train:
    # Iterate through all the elements in the list
    for i in path_list:
        # Write the current path at the end of the file
        #train.write(i + '\n')
        train.write(i)

# Create test.txt file and write 20% of data (lines) inside it.
with open('test.txt', 'w') as test:
    # Iterate through all the elements in the list
    for i in path_list_test:
        # Write the current path at the end of the file
        #test.write(i + '\n')
        test.write(i)


Next, we create the train.txt and test.txt files which contain the image paths. We first define a list, itterate over the images inside the data folder and appended their paths to the list, then split this list into 80% train and 20% test (validation). We save the 20% of test into a new list and kept the original list for the train paths only. Then we created 2 txt files, one for the train named train.txt where we saved the paths from the list, the other is for the test named test.txt where we saved the paths from the test list.

# Clone the darknet repository

In [ ]:
%cd /content/drive/MyDrive/yolov4

!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd /content/drive/MyDrive/yolov4/darknet

!make

Inside our yolov4 folder, we clone darknet from the darknet github repository, then change the current directory to the darknet folder and run the !make command which navigates to the darknet framework and compiles it.

Inside the yolov4 folder, we create a folder named custom_weights.

Inside darknet/makefile: change GPU, CUDNN, and OPENCV to 1.

In [ ]:
%cd /content/MyDrive/yolov4/custom_weight

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137


We change the current working directory to the custom_weight folder and download yolov4.conv.137 file, which contains pretrained weights for the convolutuon layers in it.

# Creating classes.names and image_data.data

In [ ]:
%cd /content/drive/MyDrive/yolov4/dataset

In [ ]:
# inside the dataset folder
class_names = ["Pneumonia", "Normal"]
train_txt_path = '/content/drive/MyDrive/yolov4/dataset/train.txt'
test_txt_path = '/content/drive/MyDrive/yolov4/dataset/test.txt'

# Create classes.names file
names_file_path = '/content/drive/MyDrive/yolov4/dataset/classes.names'
with open(names_file_path, 'w') as file:
    for name in class_names:
        file.write(name + '\n')

# Paths for the image_data.data file
names_file = '/content/drive/MyDrive/yolov4/dataset/classes.names'
backup_directory = '/content/drive/MyDrive/yolov4/darknet/backup'

# Create image_data.data file
obj_data_path = '/content/drive/MyDrive/yolov4/dataset/image_data.data'

with open(obj_data_path, 'w') as file:
    file.write(f"classes = 2\n")
    file.write(f"train = {train_txt_path}\n")
    file.write(f"valid = {test_txt_path}\n")
    file.write(f"names = {names_file}\n")
    file.write(f"backup = {backup_directory}\n")


For the classes names we have Pneumonia class and Normal class, we create a classes.names file where we saved the classes names. We then created the images_data.data file were we saved the the number of classes, the train.txt path, the test.txt path, the classes.names path, and the path of the backup directory.

Inside the darknet/cfg folder, we create a new file named yolov4_custom.cfg, then we copy the contents of the yolov4.cfg file from the darknet/cfg folder and paste it to the new yolov4_custom.cfg file we created. We then change the following parameters in yolov4_custom.cfg: batch=64, subdivision=16, max_batches= 4000, steps= 3200, 3600, filter=21, width=224, height=224, classes=2.


# Training



In [ ]:
!./darknet detector train /content/drive/MyDrive/yolov4/dataset/image_data.data /content/drive/MyDrive/yolov4/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolov4/custom_weight/yolov4.conv.137 -dont_show


We call the train command for the model and give it the images_data.data file path, the yolov4_custom.cfg file, and the yolov4.conv.137 weights file.

In [ ]:
# to continue training after it disconnects

!./darknet detector train /content/drive/MyDrive/study/darknet/data/train/image_data.data /content/drive/MyDrive/study/darknet/cfg/yolov4_custom.cfg  /content/drive/MyDrive/study/darknet/backup/yolov4_custom_last.weights -dont_show -map

In case of disconnection during training, we will continue training using this command, with the modification of the weights file were we gave it the path of the last weights of the training before it disconnected.

After training is done, the following files are created in darknet/backup folder: yolov4_custom_last.weights, yolov4_custom_1000.weights, yolov4_custom_2000.weights, yolov4_custom_3000.weights, yolov4_custom_4000.weights, yolov4_custom_final.weights.

<img src="chart.png" style="width:500px; height:500px;">


# Testing (Prediction)


In [ ]:
# Open the configuration file

with open('/content/drive/MyDrive/yolov4/darknet/cfg/yolov4_custom.cfg', 'r') as file:
    data = file.readlines()

# Iterate through the lines of the file
for i, line in enumerate(data):
    # Find lines with 'batch_normalize=1' and replace them with 'batch_normalize=0'
    if 'batch_normalize=1' in line:
        data[i] = line.replace('batch_normalize=1', 'batch_normalize=0')
    # Find lines with 'subdivisions=' and replace the value with '1'
    elif 'subdivisions=' in line:
        data[i] = 'subdivisions=1\n'
    # Find lines with 'batch=' and replace the value with '1'
    elif 'batch=' in line:
        data[i] = 'batch=1\n'

# Write the modified data back to the file
with open('/content/drive/MyDrive/yolov4/darknet/cfg/yolov4_custom.cfg', 'w') as file:
    file.writelines(data)


We change the following parameters in yolov4_custom.cfg to prepare for testing: batch=1, subdivision=1, batch_normalize=0.


In [ ]:
!./darknet detector test /content/drive/MyDrive/yolov4/dataset/image_data.data /content/drive/MyDrive/yolov4/darknet/cfg/yolov4_custom.cfg /content/drive/MyDrive/yolov4/custom_weight/yolov4_custom_final.weights /content/drive/MyDrive/yolov4/normal.jpg -out result.txt

In [ ]:
# if you get error permission denied, run this then run the previous cell

!chmod +x ./darknet

For the test command we give it the image_data.data file path, the yolov4_custom.cfg file path, the yolov4_custom_final.weights file path, and the path to the test file (from the original test file provided). We also specified the result ouput file so that the output results of the test would be saved in a txt file named result.txt.

In [ ]:
import cv2

# Load the image
image = cv2.imread('predictions.jpg')

results_file = 'result.txt'

with open(results_file, 'r') as f:
    lines = f.readlines()

# Parse each line to get bounding box coordinates, confidence, and class
results = []
for line in lines:
    # Example format: "class, confidence, x_min, y_min, x_max, y_max"
    parts = line.strip().split(',')  #read each line of the text file
    class_name = parts[0]   #get the class name
    confidence = float(parts[1]) #get the confidence 
    coords = list(map(int, parts[2:])) #get the x min , y min , x max , y max and put them into a list
    x_min, y_min, x_max, y_max = coords #then assign a variable for each value of this list
    results.append({"class": class_name, "confidence": confidence, "bbox": [x_min, y_min, x_max, y_max]})  #append the results retrived from the file to the list

# Draw bounding boxes and labels only for class "pneumonia"
for result in results:  #loop over the list
    if result["class"] == "pneumonia":   #we will draw the bounding box only if the pneumonia present
        class_name = result["class"]  #get class name "pneumonia"
        confidence = result["confidence"] #get the confidence
        bbox = result["bbox"] #get the bounding box (xmin,ymin,xmax,ymax)
        x_min, y_min, x_max, y_max = bbox  #assign a variable for each value in the bounding box
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  #draw the rectangle on the image
        label = f"{class_name}: {confidence:.2f}"  #label the rectangle with the class name and the confidence
        cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2) #put the label and confidence on the image

# Display or save the image with annotations
cv2.imshow('Image with Annotations', image)  
cv2.waitKey(0)
cv2.destroyAllWindows()

# To save the image:
cv2.imwrite('annotated_image.jpg', image)


We use OpenCV to load the predicted image , read detection results from the result.txt file, and annotate the image with bounding boxes for class "Pneumonia". We extract the (class name, confidence, x_min, y_min, x_max, y_max) then store them in a list. The annotated image is displayed using a window and saved to a file named annotated_image.jpg.